In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
from numpy import linalg as LA
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

In [2]:
n      = 1.33
c      = 299792458/n
R      = 7.3/2 #--------------------- radius of tank
v      = n * c
times  = linspace(7e-10,1e-7,100000)

c1       = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
c2       = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
c3       = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
c4       = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
xA,xB,h  = 3, -3, 1.5

plotme = 'x vs t'                                                                                          #------PARAM

A   = array( [xA, sqrt(R**2-xA**2), h] )
B   = array( [xB, -sqrt(R**2-xB**2), h] )
AB  = B-A
nAB = LA.norm(AB)
den = c*c - v*v
print('A : ', A)
print('B : ', B)

A :  [3.         2.07906229 1.5       ]
B :  [-3.         -2.07906229  1.5       ]


In [4]:
%matplotlib
plt.figure()

detector_coordinates = [c1,c2,c3,c4]
colors    = ['r','b','g','c']
detectors = [1,2,3,4]
plotme    = 'b vs t'

for D, color, detector in zip(detector_coordinates, colors, detectors):
    
    AD    = D - A
    L     = LA.norm(AD)
    alpha = arccos((sum(AD*AB))/(L*nAB)) # OR use alpha1 = arccos( (L*L+x*x-k*k)/(2*x*L) )
    
    T, aX1, aX2, aB1, aB2 = [], [], [], [], []
    #leb = left_end_brightness(A,B,h,v,c)
    
    num_xc = L*cos(alpha)*(c*c-v*v) + sqrt(-c**4*L**2*sin(alpha)**2 + c**2*L**2*v**2*sin(alpha)**2)
    den    = (c**2-v**2)
    xc     = (num_xc/den)
    print('First RID distance from A: {} m'.format(xc))
    if xc>nAB:
        print('Sorry! First RID is outside the tank. Not processing this case further.')
    
    for iii,t in enumerate(times):

        x     = v*t
        ratio = x/nAB
        X     = array([ (1-ratio)*A[0] + ratio*B[0] , (1-ratio)*A[1] + ratio*B[1] , h   ])
        AX    = X - A
        DX    = X - D
        k     = LA.norm(DX)

        t1    = t
        t2    = k/c
        tt    = t1 + t2    
        
        x1    = (c*c*tt*v-L*v*v*cos(alpha)+sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den
        x2    = (c*c*tt*v-L*v*v*cos(alpha)-sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den

        v1    = ( (c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den
        v2    = ((-c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den 

        k1    = sqrt(L*L + x1*x1 - 2*L*x1*cos(alpha))
        k2    = sqrt(L*L + x2*x2 - 2*L*x2*cos(alpha))
        
        beta1 = pi - arccos((x1*x1 + k1*k1 - L*L)/(2*x1*k1))
        beta2 = pi - arccos((x2*x2 + k2*k2 - L*L)/(2*x2*k2))
        
        v1t   = v1*sin(beta1)
        v2t   = v2*sin(beta2)

        omega1 = v1t / k1
        omega2 = v2t / k2

        b1 = abs(omega1 / (k1**2))
        b2 = abs(omega2 / (k2**2))

        aX1.append(x1)
        aX2.append(x2)
        aB1.append(b1)
        aB2.append(b2)
        T.append(tt)

    aX1, aX2, aB1, aB2, T = array(aX1), array(aX2), array(aB1), array(aB2), array(T)
    Tn  = T * 1e+9
    TT  = (Tn-Tn.min())
    
    if plotme != 'b vs t':
        plt.plot(TT, aX1, c=color, ls='--')   # 1 == towards A    2 == towards B
        plt.plot(TT, aX2, c=color, ls='-' , label='Detector {}'.format(detector))
        plt.axhline(xc, c='k', ls=':')
        plt.text(35, round(xc,2)+0.085, r'$x_c = {} \; m$'.format(round(xc,2)), fontsize=16)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'image distance from the entry point $x_{\pm}$ (m)'       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(0,nAB)
        plt.xlim(0,40)
    else:
        plt.plot(TT, aB1, c=color, ls='--')
        plt.plot(TT, aB2, c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{ground}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg
First RID distance from A: 1.9393690371488181 m
First RID distance from A: 0.7871005066698564 m
First RID distance from A: 3.389519992729894 m
First RID distance from A: 0.03889150295637708 m
